## Load necessary modules

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join( 'snapshots', 'example_inference.hdf5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'labelName'}

## Define Utility Functions

In [ ]:
def draw_box(image, box, color, thickness=2):
    """ Draws a box on an image with a given color.

    # Arguments
        image     : The image to draw on.
        box       : A list of 4 elements (x1, y1, x2, y2).
        color     : The color of the box.
        thickness : The thickness of the lines to draw a box with.
    """
    b = np.array(box).astype(int)
    cv2.rectangle(image, (b[0], b[1]), (b[2], b[3]), color, thickness, cv2.LINE_AA)
    
def draw_caption(image, box, caption,):
    """ Draws a caption above the box in an image.

    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    b = np.array(box).astype(int)
    # cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, lineType=cv2.LINE_AA)
    b_y = int((b[1]+b[3])/2)
    #cv2.putText(image, caption, (b[0], b_y+j*80), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 0), 3, lineType=cv2.LINE_AA)
    #cv2.putText(image, caption, (b[0], b_y+j*80), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (255, 255, 255), 3, lineType=cv2.LINE_AA)
    cv2.putText(image, caption, (b[0], b_y), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 0), 1, lineType=cv2.LINE_AA)
    cv2.putText(image, caption, (b[0], b_y), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 255, 255), 1, lineType=cv2.LINE_AA)

## Run detection on example

In [ ]:
import glob
import os
import cv2
test_images_path = '' # path to test images
test_images = glob.glob(test_images_path + "*.jpeg") + glob.glob(test_images_path + "*.png") +  glob.glob(test_images_path + "*.jpg")


In [ ]:
for i in test_images:
# load image
    image = read_image_bgr(i)
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)
    #print(boxes[0].shape,scores[0].shape,labels[0].shape)
    selected_indices = tf.image.non_max_suppression( boxes[0], scores[0], max_output_size=300, iou_threshold=0.1)
    #selected_indices = selected_indices.eval(session=sess)
    selected_boxes = tf.gather(boxes[0], selected_indices)
    selected_scores = tf.gather(scores[0], selected_indices)
    selected_labels = tf.gather(labels[0], selected_indices)
    selected_boxes = selected_boxes.eval(session=tf.Session())
    selected_scores = selected_scores.eval(session=tf.Session())
    selected_labels = selected_labels.eval(session=tf.Session())
    #print(selected_boxes.shape,selected_scores.shape,selected_labels.shape)
    # correct for image scale
    selected_boxes /= scale
    #boxes = non_max_suppression_fast(boxes[0],0.5)    
    #boxes /= scale
    # visualize detections
    for box, score, label in zip(selected_boxes, selected_scores, selected_labels):
    #for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        #color = label_color(label)
        color = label_color(label+1)

        b = box.astype(int)
        draw_box(draw, b, color=color, thickness=2)

        caption_ori = "{:.3f}".format(score)
        #for j, line in enumerate(caption_ori.split('/')): 
        draw_caption(draw, b, caption_ori,)
        
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()
    # now save detection results to folder 'testResults'
    output_folder = 'testResults'
    fp,fn = os.path.split(i)
    os.makedirs(output_folder, exist_ok=True)
    plt.imsave(os.path.join(output_folder,'{}'.format(fn)),draw)


## End